## Library import

In [ ]:
import numpy as np
import scipy as sc
import math
import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import pandas
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
import numpy as np
from scipy.optimize import minimize

## Linear Regression

Start with y = exp(x) + normal noise, $x \in [0,1)$

In [ ]:
N = 20
h = 1./N
noise = 0.1
x = np.arange(N)*h
yexact = np.exp(x)
y = yexact+noise*np.random.normal(size=x.shape)

Do linear regression using the lecture formula

In [ ]:
x2sum = np.sum(x**2)
xsum = np.sum(x)
ysum = np.sum(y)
xysum = np.sum(x*y)
A = [[x2sum,xsum],[xsum,N]]
b = [xysum, ysum]
u = np.linalg.solve(A, b)
mlin = u[0]
blin = u[1]
print 'm: %10.6f b: %10.6f' % (mlin,blin)
yfit = mlin*x+blin
plt.figure(1)
plt.plot(x,yexact,x,y,'ro',x,yfit)
plt.show()

Use the numpy least squares routine, that probably does SVD 

In [ ]:
xmatrix = np.zeros((N,2))
xmatrix[:,0] = x
xmatrix[:,1] = np.zeros(N)+1
m, b = np.linalg.lstsq(xmatrix,y)[0]
print 'm: %10.6f b: %10.6f' % (m,b)

Now do a quadratic fit

In [ ]:
xmatrix = np.zeros((N,3))
xmatrix[:,0] = x**2
xmatrix[:,1] = x
xmatrix[:,2] = np.zeros(N)+1
q, l, b = np.linalg.lstsq(xmatrix,y)[0]
print 'q: %10.6f l: %10.6f b: %10.6f' % (q,l,b)
yfit = q*x**2+l*x+b
plt.figure(1)
plt.plot(x,yexact,x,y,'ro',x,yfit)
plt.show()

If you were a statistician you would know what all this means...

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print 'm: %10.6f b: %10.6f' % (slope,intercept)
print "r-squared:", r_value**2
print 'p value:', p_value # two-sided p-value for a hypothesis test whose null hypothesis is that the slope is zero.

and this might need a graduate statistics degree... 

In [ ]:
data = pandas.DataFrame({'x2': x**2, 'x': x, 'y': y})

# Fit the model
model = ols("y ~ x2 + x", data).fit()

# Print the summary
print(model.summary())

# Fitted parameters
print("\nParameter estimates:")
print(model._results.params)

# Peform analysis of variance on fitted linear model
anova_results = anova_lm(model)
print('\nANOVA results')
print(anova_results)


### The effect of an outlier -- leads to an exercise

In [ ]:
y2 = y
y2[N-1] = 5
x2sum = np.sum(x**2)
xsum = np.sum(x)
ysum = np.sum(y2)
xysum = np.sum(x*y2)
A = [[x2sum,xsum],[xsum,N]]
b = [xysum, ysum]
u = np.linalg.solve(A, b)
mlin = u[0]
blin = u[1]
print 'm: %10.6f b: %10.6f' % (mlin,blin)
yfit = mlin*x+blin
plt.figure(1)
plt.plot(x,yexact,x,y2,'ro',x,yfit)
plt.show()

## Nonlinear System Example

The discretization of $-u^{\prime \prime}+u+u^3 = f$

preliminaries

In [ ]:
N = 64;
pi = math.pi
h = 2*pi/N
fact = 1/h**2
A = csr_matrix((N,N))
x = np.arange(N)*h
rhs = np.zeros(N)
for j in range(0,N):  
    rhs[j] = math.exp(math.cos(x[j]))*(math.cos(x[j])+math.cos(x[j])**2)

function to return the Jacobian matrix

In [ ]:
def Jacobian(A,u,fact,rhs):
    vres = np.zeros(N)
    for j in range(0,N):  # loop by matrix row
        A[j,j] = 2*fact+1+3*u[j]**2 # diagonals
        if j==0:
            jm = N-1
        else:
            jm = j-1
        if j==N-1:
            jp = 0
        else:
            jp = j+1
        A[j,[jp,jm]] = -fact # off diagonals
        vres[j] = 2*fact*u[j] + u[j]**3 + u[j] - fact*(u[jm]+u[jp])-rhs[j]
    return[A, vres]

The Newton iteration

In [ ]:
tol = 1e-6
res = 2*tol 
count = 0 
u = np.zeros(N)
A,vres = Jacobian(A,u,fact,rhs)
print 'Iteration count and max norm residual:'
while res > tol and count < 8:
    count = count +1
    delta = spsolve(A, vres)
    u = u - delta
    A,vres = Jacobian(A,u,fact,rhs)
    res = np.amax(np.absolute(vres))
    print count, res
plt.figure(1)
plt.plot(x,u) 
plt.show()
    

## Optimization Example

Minimization of the Rosenbrock function

Code taken directly from the scipy web pages 

In [ ]:
def rosen(x):
     return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
    
def rosen_der(x):
     xm = x[1:-1]
     xm_m1 = x[:-2]
     xm_p1 = x[2:]
     der = np.zeros_like(x)
     der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
     der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
     der[-1] = 200*(x[-1]-x[-2]**2)
     return der

def rosen_hess(x):
     x = np.asarray(x)
     H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
     diagonal = np.zeros_like(x)
     diagonal[0] = 1200*x[0]**2-400*x[1]+2
     diagonal[-1] = 200
     diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
     H = H + np.diag(diagonal)
     return H

In [ ]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='Newton-CG', jac=rosen_der, hess=rosen_hess, options={'xtol': 1e-8, 'disp': True})
print res.x

In [ ]:
res = minimize(rosen, x0, method='Newton-CG', jac=rosen_der, options={'xtol': 1e-8, 'disp': True})

In [ ]:
res = minimize(rosen, x0, method='BFGS', jac=rosen_der, options={'disp': True})

In [ ]:
res = minimize(rosen, x0, method='BFGS', options={'disp': True})

In [ ]:
res = minimize(rosen, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})